In [1]:
pip install keras_ocr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 67.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.5 MB/s eta 0:00:00


## Import Library

In [2]:
import os
import shutil
import csv
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import string
import imgaug


import cv2
import keras_ocr
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the training and validation base directories
img_dir = '/content/drive/MyDrive/bg udh bg/image'
validation_dir = '/content/drive/MyDrive/bg udh bg/eval_set'
img_csv = '/content/drive/MyDrive/bg udh bg/image/DataTrain.csv'

In [5]:
img_df = pd.read_csv(img_csv, sep = ';', index_col=0)
img_df.head()

,Vehicleregistrationplate,NameofFile
0,A7814,DataTrain1.png
1,B1074QO,DataTrain2.png
2,B1031QO,DataTrain3.png
3,B187EDA,DataTrain4.png
4,B1089VD,DataTrain5.png


In [6]:
def img_path(dir, df):
  image_paths = []
  for index, row in df.iterrows():
      image_name = row['NameofFile']
      image_path = os.path.join(dir, image_name)
      image_paths.append(image_path)

  return image_paths

## Pre-trained

In [7]:
image_paths = img_path(img_dir, img_df)

In [8]:
labels = img_df.Vehicleregistrationplate.tolist()
labels_lower = [s.lower() for s in labels]

In [18]:
len(labels)

800

In [15]:
formatted_data = [(image_path, None, labels) for image_path, label in zip(image_paths, labels)]

In [ ]:
formatted_data

In [10]:
detector = keras_ocr.detection.Detector(weights='clovaai_general')
recognizer = keras_ocr.recognition.Recognizer()
recognizer.compile()
for layer in recognizer.backbone.layers:
    layer.trainable = False

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [11]:
batch_size = 10
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.GammaContrast(gamma=(0.25, 3.0)),
])

train_labels, validation_labels = train_test_split(formatted_data, test_size=0.2, random_state=42)
(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=recognizer.model.input_shape[1],
            width=recognizer.model.input_shape[2],
            alphabet=recognizer.alphabet,
            augmenter=augmenter
        ),
        len(labels) // batch_size
    ) for labels, augmenter in [(train_labels, augmenter), (validation_labels, None)]
]

In [12]:
training_gen, validation_gen = [
    recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size,
        lowercase=True
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

In [ ]:
recognition_batch_size = 10
recognizer_basepath = os.path.join(img_dir, f'recognizer_{datetime.datetime.now().isoformat()}')

In [ ]:
recognizer.training_model.fit(
    training_gen,
    epochs=10,
    steps_per_epoch=training_steps,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=25),
        tf.keras.callbacks.CSVLogger(f'{recognizer_basepath}.csv', append=True),
        tf.keras.callbacks.ModelCheckpoint(filepath=f'{recognizer_basepath}.h5')
    ],
    validation_data=validation_gen,
    validation_steps=validation_steps,
    workers=0,
    batch_size=recognition_batch_size
)

640 / 640 instances have illegal characters.
